# Verify ECDSA Secp256K1 Signature

In [40]:
from src.util import pad_base64
import base64
import hashlib

sig = "sw_xxlA6r19a4p3v9aMAESqtMtQOTkvAPa3fpHDTe1MqPrLnkKNyrKl2a9IulJioLYSmoDa-qV8kQoBHRNrVGg"
sig = pad_base64(sig)

decoded_sig = base64.urlsafe_b64decode(sig)

r = int.from_bytes(decoded_sig[:32], byteorder="big")
s = int.from_bytes(decoded_sig[32:], byteorder="big")

sig_obj = {
    "r": r,
    "s": s
}

did = "did:ebsi:znHeZWvhAK2FK2Dk1jXNe7m"
did_doc = {
  "@context": "https://www.w3.org/ns/did/v1",
  "id": "did:ebsi:znHeZWvhAK2FK2Dk1jXNe7m",
  "verificationMethod": [
    {
      "id": "did:ebsi:znHeZWvhAK2FK2Dk1jXNe7m#keys-1",
      "type": "Secp256k1VerificationKey2018",
      "controller": "did:ebsi:znHeZWvhAK2FK2Dk1jXNe7m",
      "publicKeyJwk": {
        "kty": "EC",
        "crv": "secp256k1",
        "x": "Ou6y1zrJBeVnpV739kcTyez7RmQZFYg3F9bWGm6V5dQ",
        "y": "jOq6B8CsOxoXj-WXAGY28PH0Ype1x6bnOB6_YOo3lK0"
      }
    }
  ],
  "authentication": [
    "did:ebsi:znHeZWvhAK2FK2Dk1jXNe7m#keys-1"
  ],
  "assertionMethod": [
    "did:ebsi:znHeZWvhAK2FK2Dk1jXNe7m#keys-1"
  ]
}

authenticator = did_doc["verificationMethod"][0]

pub_key_x = authenticator["publicKeyJwk"]["x"]
pub_key_y = authenticator["publicKeyJwk"]["y"]

pub_key_x_bytes = base64.urlsafe_b64decode(pad_base64(pub_key_x))
pub_key_y_bytes = base64.urlsafe_b64decode(pad_base64(pub_key_y))

int_x = int.from_bytes(pub_key_x_bytes, byteorder="big")
int_y = int.from_bytes(pub_key_y_bytes, byteorder="big")

from eth_keys import KeyAPI

keys = KeyAPI('eth_keys.backends.CoinCurveECCBackend')

signature = keys.Signature(vrs=(0, r, s))

from coincurve.keys import PublicKey

public_key_cc = PublicKey.from_point(int_x, int_y)

public_key_cc_compressed_bytes = public_key_cc.format()

public_key_ethk = keys.PublicKey.from_compressed_bytes(public_key_cc_compressed_bytes)

data_jwt = "eyJhbGciOiJFUzI1NksiLCJ0eXAiOiJKV1QiLCJraWQiOiJodHRwczovL2FwaS5jb25mb3JtYW5jZS5pbnRlYnNpLnh5ei90cnVzdGVkLWFwcHMtcmVnaXN0cnkvdjIvYXBwcy8weDU1OWM0ZjMyZGMzNTU2NmU0YjkyYjY5NzQ5OWMzOGYzODQ3YTZjNTNmODM0NDgyMWMyNDM1NGVhZDFmMmFiMWUifQ.eyJpYXQiOjE2NTI4MDc2NjUsImV4cCI6MTY1MjgwNzk2NSwiaXNzIjoiZGlkOmVic2k6em5IZVpXdmhBSzJGSzJEazFqWE5lN20iLCJzY29wZSI6Im9wZW5pZCBkaWRfYXV0aG4iLCJyZXNwb25zZV90eXBlIjoiaWRfdG9rZW4iLCJyZXNwb25zZV9tb2RlIjoicG9zdCIsImNsaWVudF9pZCI6Imh0dHBzOi8vYXBpLmNvbmZvcm1hbmNlLmludGVic2kueHl6L2F1dGhvcmlzYXRpb24vdjEvc2lvcC1zZXNzaW9ucyIsInJlZGlyZWN0X3VyaSI6Imh0dHBzOi8vYXBpLmNvbmZvcm1hbmNlLmludGVic2kueHl6L2F1dGhvcmlzYXRpb24vdjEvc2lvcC1zZXNzaW9ucyIsIm5vbmNlIjoiMGQ0M2QyYmUtZTEyMi00NDk3LWI4NjgtNDEwYzg2MzViMzE5IiwiY2xhaW1zIjp7ImlkX3Rva2VuIjp7InZlcmlmaWVkX2NsYWltcyI6eyJ2ZXJpZmljYXRpb24iOnsidHJ1c3RfZnJhbWV3b3JrIjoiRUJTSSIsImV2aWRlbmNlIjp7InR5cGUiOnsidmFsdWUiOiJ2ZXJpZmlhYmxlX2NyZWRlbnRpYWwifSwiZG9jdW1lbnQiOnsidHlwZSI6eyJlc3NlbnRpYWwiOnRydWUsInZhbHVlIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiVmVyaWZpYWJsZUF1dGhvcmlzYXRpb24iXX0sImNyZWRlbnRpYWxTY2hlbWEiOnsiaWQiOnsiZXNzZW50aWFsIjp0cnVlLCJ2YWx1ZSI6Imh0dHBzOi8vYXBpLmNvbmZvcm1hbmNlLmludGVic2kueHl6L3RydXN0ZWQtc2NoZW1hcy1yZWdpc3RyeS92MS9zY2hlbWFzLzB4MjhkNzY5NTQ5MjRkMWM0NzQ3YTRmMWY5ZTNlOWVkYzljYTk2NWVmYmY4ZmYyMGU0MzM5YzJiZjIzMjNhNTc3MyJ9fX19fX19fX0.sw_xxlA6r19a4p3v9aMAESqtMtQOTkvAPa3fpHDTe1MqPrLnkKNyrKl2a9IulJioLYSmoDa-qV8kQoBHRNrVGg"

from src.did_jwt import decode_jwt

decoded_jwt = decode_jwt(data_jwt)

msg_bytes = hashlib.sha256(decoded_jwt["data"].encode('utf-8')).digest()

verify = keys.ecdsa_verify(msg_bytes, signature, public_key_ethk)

print(verify)


True


In [41]:
import time
import math

print(math.floor(time.time()))

1652817041


In [2]:
from src.ebsi_client import EbsiClient

client = EbsiClient()
public_key_jwk = client.eth.public_key_to_jwk()

public_key_jwk = {
    "kty": public_key_jwk.get("kty"),
    "crv": public_key_jwk.get("crv"),
    "x": public_key_jwk.get("x"),
    "y": public_key_jwk.get("y")
}

print(public_key_jwk)

{'kty': 'EC', 'crv': 'secp256k1', 'x': '2NsMAtIUNr3XAvat9x8KkW8iBAsOwaIdWyzdM7SUEok', 'y': '7HE2YQVAhG4oAjHVqxCZnWcqfPJkXCp2m3CEqYKKuHA'}
